In [ ]:
# Import packages for calculations
#piplite presence will be shown with these statememnts
try:    
    import piplite
except ModuleNotFoundError:
    piplite=None
# Use these packages for download of data
if piplite:
    await piplite.install("mplfinance")
    await piplite.install("finta")
    await piplite.install("ipympl")
    await piplite.install("ipywidgets")
    await piplite.install("jinja2")
    try:
        await piplite.install("/pypi/backtrader-1.9.76.123-py3-none-any.whl")
    except ValueError as e:
        print("localExampleEnv-failed")
        try: 
            await piplite.install("http://localhost/staticcollected/_output/pypi/backtrader-1.9.76.123-py3-none-any.whl")
        except ValueError as e:
            print("localProd00env-failed")
            await piplite.install("http://www.stockwhiz.in/staticcollected/_output/pypi/backtrader-1.9.76.123-py3-none-any.whl")
    # This is for the fetch in piplite
    import js 
    from pyodide.ffi import to_js
    from js import Object
else:
    import yfinance
#3 Imports of all the files
import json
import pandas as pd
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import mplfinance as mpf
import scipy as sp
from IPython.display import display, Markdown
import backtrader as bt
import finta

%matplotlib widget
pd.options.display.max_columns = None

---

# 03 Simple moving average crossover 

## About
A simple moving average trading occurs whenever the price of the stock moves over or below the average price of the stock for a said period of days

## Usage
Click the $>>$ icon above to run this script
Or run each of the cells individually as a Jupyter Notebook

## What?
This script provides you with an ability to determine a simple change in the moving average of the stock. 
Note: *Feel free to move around for appropriate day count at the moving average*

---

## Grabbing data

***The data being provisioned is only for educational purposes***


*Note: You can use the free service limits usage to only 100 symbols per minute. Tampering with this system will result in discontinuation of the service*

## Adding symbols

Please feel free to add any **NSE SYMBOL** to the below list

In [ ]:
# The stock to backtest 
symbols = "ITC"

In [ ]:
# Download historical data for strategy (1 year/s)
corsprox = "https://corsproxy.io/?"
apiBaseURL = "https://query1.finance.yahoo.com/v8/finance/chart/"
rangeOfData = "12mo"
intervalOfData = "1d"
apiSymbol = []
for symbol in symbols.split(","):
    apiSymbol.append(symbol+".NS")
stockPandasTot = {}
for apiSymbolIndivi in apiSymbol:
    print(apiSymbolIndivi)
    apiCompleteURL = corsprox+apiBaseURL+apiSymbolIndivi+\
        "?range="+rangeOfData+"&interval="+intervalOfData
    options = { "mode": "cors",
                "credentials":"omit",
                "headers": {'Accept': 'application/json',
                            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
                            'Origin':'https://yahoo.com',
                            }
    }
    if piplite:
        resp = await js.fetch(apiCompleteURL, to_js(options, dict_converter=Object.fromEntries))
        text = await resp.text()
        pan = pd.read_json(text)
        a_hist = pan['chart'][1][0]
        stockPandas = pd.DataFrame(a_hist['indicators']['quote'][0],
                               index=pd.to_datetime(a_hist['timestamp'],unit='s'))
        stockPandas = stockPandas.reindex(columns=['open','high','low','close','volume'])
        stockPandas = stockPandas[::-1]
    else:
        stockPandas = yfinance.download(apiSymbolIndivi,period=rangeOfData,interval=intervalOfData,progress=False)
        stockPandas = stockPandas.round(2).rename(columns={
                                                    'Open': 'open',
                                                    'High': 'high',
                                                    'Low': 'low',
                                                    'Close': 'close',
                                                    'Adj Close': 'adj close',
                                                    'Volume': 'volume'
                                                })
        stockPandas = stockPandas[::-1]
    stockPandasTot[apiSymbolIndivi] = stockPandas

## Simple moving average crossover

[![Moving Average crossover](https://th.bing.com/th/id/R.9de5e9b8cde12f94f618278cdd074c92?rik=yxyy%2bEHrN%2fP4ZQ&riu=http%3a%2f%2fnewtraderu.com%2fwp-content%2fuploads%2f2013%2f01%2fMovingAverage2.gif&ehk=76K1eC4aQMP%2fm4kgMZf420B2bk0Vpfzkhr2WvshtJPA%3d&risl=&pid=ImgRaw&r=0)](https://th.bing.com/th/id/R.9de5e9b8cde12f94f618278cdd074c92?rik=yxyy%2bEHrN%2fP4ZQ&riu=http%3a%2f%2fnewtraderu.com%2fwp-content%2fuploads%2f2013%2f01%2fMovingAverage2.gif&ehk=76K1eC4aQMP%2fm4kgMZf420B2bk0Vpfzkhr2WvshtJPA%3d&risl=&pid=ImgRaw&r=0)

One can choose the lengths of periods which they are interested in for crossovers
$$
L_1 = 5 \\
L_2 = 15 
$$

The moving average crossover signal is formed when 0th day smaller moving average is greater than the larger moving average
- No previous trend required
- $SMA_1 > SMA_2$
- dates weightages are till day 3 as follows
    - With weights as follows
    - $ day 0 : 15 $
    - $ day 1 : 10 $
    - $ day 2 : 5 $
    - $ day 3 : 2 $

In [ ]:
# Setup variables for calculations (Single Cross Over)
## Variables
# Your choice of moving average days
L_1 = 5
# Your choice of moving average days
# Please ensure L_2 is greater than L_1
L_2 = 15
# Number of days checked [Can we push this dates checked to different cell?]
datesChecked = len(stockPandas['close'])
# weights of each individual day (Shape in reshape function should be matched. )
weights = np.array([15, 10, 5, 2])
# Adding additional weights, will be helfull only while checking.
weights = np.append(weights, np.ones((datesChecked-4,1)))
signalBullDict = {}

---
## Backtesting
This portion is to conduct backtest on the strategy mentioned above.

In [ ]:
# Setup Strategy for backtest (Interactivity to be improved)
class TestStrategy(bt.Strategy):

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        #Some indicators
        self.sma1 = bt.indicators.SimpleMovingAverage(self.datas[0],period = 15)
        self.sma2 = bt.indicators.SimpleMovingAverage(self.datas[0],period = 50)

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:
            if self.sma1[-1] < self.sma2[-1]:
                if self.sma1[0] > self.sma2[0]:
                    # Keep track of the created order to avoid a 2nd order
                    self.order = self.buy(size=100)

        else:

            # Already in the market ... we might sell
            if self.sma1 < self.sma2:
                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell(size=100)

In [ ]:
# Run the strategy (Interactivity improvements)
cerebro = bt.Cerebro()
    
cerebro.addstrategy(TestStrategy)
    
data = bt.feeds.YahooFinanceCSVData(dataname="data.txt",fromdate=datetime(2000, 1, 1),todate=datetime(2000, 12, 31))
data2 = bt.feeds.PandasData(dataname=stockPandasTot["ITC.NS"][::-1])
cerebro.adddata(data2)

cerebro.broker.setcash(100000)

cerebro.broker.setcommission(commission=0.001)

print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

cerebro.run()

print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

In [ ]:
# Plot how strategy fares (Interactivity improvements)
cerebro.plot(style="candlestick",figsize=(3,1))